## List S3 Buckets and Create an S3 Bucket

In [6]:
import boto3

s3 = boto3.client('s3')

bucket_name = 'somerandomashustorage'

In [4]:
response = s3.list_buckets()
response

{'ResponseMetadata': {'RequestId': 'J88Y4PC655VGQTST',
  'HostId': '2ojMbcQc2sfJNqLm+NqlPxiNOTw7LNhQ5sYOFe76HsuH9Fy3zDRk+GQbs2pS2SPehxothSpoKn8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '2ojMbcQc2sfJNqLm+NqlPxiNOTw7LNhQ5sYOFe76HsuH9Fy3zDRk+GQbs2pS2SPehxothSpoKn8=',
   'x-amz-request-id': 'J88Y4PC655VGQTST',
   'date': 'Wed, 30 Oct 2024 20:20:28 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [],
 'Owner': {'DisplayName': 'ashutosh.shankar.a',
  'ID': 'aaab05ff2b0bbe65c1e12ce2025cc296b1472b041cea20392b9e120e82a3864b'}}

In [7]:
def create_bucket(bucket_name):
    s3.create_bucket(Bucket=bucket_name)
    print('Bucket is created')

create_bucket(bucket_name)

Bucket is created


## Upload a Single File to S3

In [8]:
import os
def upload_file(file_path,object_name=None):
    if object_name == None:
        object_name = os.path.basename(file_path)
    s3.upload_file(file_path, bucket_name, object_name)
    

In [9]:
upload_file('data/index.html')

In [10]:
upload_file('data/index.html', 'somefile.html')

## List all objects in a bucket

In [11]:
s3.list_objects_v2(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'HP9A6VRQC7M30RSA',
  'HostId': 'vh+fFjJSOiTWZ7Ggd5vqwc4kI5JpwcomCCWq7qp5OMjvdkCnvJfOeK2dh1b/2O9SRontr7ELoRQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'vh+fFjJSOiTWZ7Ggd5vqwc4kI5JpwcomCCWq7qp5OMjvdkCnvJfOeK2dh1b/2O9SRontr7ELoRQ=',
   'x-amz-request-id': 'HP9A6VRQC7M30RSA',
   'date': 'Wed, 30 Oct 2024 20:39:55 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'index.html',
   'LastModified': datetime.datetime(2024, 10, 30, 20, 37, 27, tzinfo=tzutc()),
   'ETag': '"be89714840f63aec115562d5f747fca9"',
   'Size': 43,
   'StorageClass': 'STANDARD'},
  {'Key': 'somefile.html',
   'LastModified': datetime.datetime(2024, 10, 30, 20, 38, 20, tzinfo=tzutc()),
   'ETag': '"be89714840f63aec115562d5f747fca9"',
   'Size': 43,
   'StorageClass': 'STANDARD'}],
 'Name': 'somerandomashustora

In [13]:
def list_objects():
    response = s3.list_objects_v2(Bucket =bucket_name)
    for _ in response['Contents']:
        print(_['Key'])
        
list_objects()

index.html
somefile.html


## Download S3 File to Local System

In [14]:
def download_file(object_name, file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))
    s3.download_file(bucket_name, object_name, file_path)
    
download_file('index.html', 'data_downlaods/index.html')

## Upload folder to S3

In [16]:
for root, dirs, files in os.walk('data'):
    print(root,dirs,files)

data [] ['panda-1236875_640.jpg', 'index.html', 'water-5393919_1280.jpg']


In [20]:
def upload_directory(directory_path, s3_prefix):
    for root, dirs, files in os.walk('data'):
        for file in files:
            file_path = os.path.join(root, file)
            relpath = os.path.relpath(file_path, directory_path)
            # print(relpath)
            s3_key = os.path.join(s3_prefix, relpath)
            # print(s3_key)
            
            s3.upload_file(file_path,bucket_name,s3_key)

upload_directory('data', 's3_data')
    

## Download S3 Folder to Local System

In [21]:
local_path = 's3_download_1'
s3_prefix = 's3_data'

def download_dir(local_path, s3_prefix):
    os.makedirs(local_path, exist_ok=True)
    paginator = s3.get_paginator('list_objects_v2')
    for result in paginator.paginate(Bucket=bucket_name, Prefix=s3_prefix):
        if 'Contents' in result:
            for key in result['Contents']:
                s3_key = key['Key']

                local_file = os.path.join(local_path, os.path.relpath(s3_key, s3_prefix))
                # os.makedirs(os.path.dirname(local_file), exist_ok=True)

                s3.download_file(bucket_name, s3_key, local_file)



download_dir(local_path, s3_prefix)

## Delete All Files in an S3 Bucket

In [22]:
def delete_objects():
    response = s3.list_objects_v2(Bucket=bucket_name)

    if 'Contents' in response:

        for obj in response['Contents']:
            s3.delete_object(Bucket=bucket_name, Key=obj['Key'])



delete_objects()

In [23]:
s3.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': '3WZEDMP944C0E2N6',
  'HostId': 'Izz9UklE9nYxJHNnbWE6g4LxDD8UueI0MSxaEzcsQNF6hGkfRH8Vt9H1WkW9Rse2P3hCSTnX6ezY1xTbUYEmFyRrrxR/uz2e29OSSYXkP6Q=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'Izz9UklE9nYxJHNnbWE6g4LxDD8UueI0MSxaEzcsQNF6hGkfRH8Vt9H1WkW9Rse2P3hCSTnX6ezY1xTbUYEmFyRrrxR/uz2e29OSSYXkP6Q=',
   'x-amz-request-id': '3WZEDMP944C0E2N6',
   'date': 'Wed, 30 Oct 2024 21:27:52 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}